In [5]:
import numpy as np
import math
import copy
import torch
LARGEPRIME = 2**61-1
# torch.random.manual_seed(42)

In [1]:
import logging

In [16]:
def create_stream(n,m,w):
    assert m > w
    np.random.seed(42)
    stream = np.random.randint(1,n+1,size=m)
    exactL2w = None
#     exactL2 = np.linalg.norm(stream, ord=2)
#     exactL2w = np.linalg.norm(stream[-w:], ord=2)    
#     logging.info('exact L2 of whole stream {:0.2f}'.format(exactL2))
#     logging.info('exact L2 of last windowed stream {:0.2f}'.format(exactL2w))
    return stream, exactL2w

In [20]:
n, m , w =100, 201, 200
stream, _ = create_stream(n,m,w)

In [31]:
int(np.log10(500))

2

In [10]:
from collections import Counter

In [21]:
c = Counter(stream)

In [26]:
def get_freqList(stream):
    c = Counter(stream)    
    freqList=[]
    for i in range(1, n+1):
        freqList.append(c[i])
    assert len(freqList) == n
    return freqList

In [29]:
from dataset import *

In [ ]:
exactNorm = np.linalg.norm(freqList, ord=2)    


In [30]:
uniform_sampling(freqList, w, sRate=0.1)

37.28270376461449

In [27]:
freqList = get_freqList(stream)

In [68]:
def normFn(name):
    """
    choose between 'L1', 'L2'...'Lp', 'Tk'
    """
    if name[0] == 'L':
        try: p = int(name[1:]) 
        except: raise 'enter # for p in Lp'
        return lambda x: sum([abs(i)**p for i in x])**(1.0/p)
    elif name[0] == 'T':
        try: k = int(name[1:])
        except: raise 'enter # for k in Tk'
        return lambda x: sum(np.sort([abs(i) for i in x])[::-1][:k])

In [71]:
aa = normFn('T10')

In [72]:
aa(stream)

969

In [75]:
norms

NameError: name 'norms' is not defined

In [ ]:
streamTr=

In [74]:
torch.argsort(streamTr)

NameError: name 'streamTr' is not defined

In [133]:
n, m , w =100, 1000, 250
assert m > w
np.random.seed(42)
stream = np.random.randint(1,n,size=m)
exactL2 = np.linalg.norm(stream, ord=2)
exactL2w = np.linalg.norm(stream[-w:], ord=2)

In [134]:
print('exact L2 of whole stream {:0.2f}'.format(exactL2))
print('exact L2 of last windowed stream {:0.2f}'.format(exactL2w))

exact L2 of whole stream 1820.19
exact L2 of last windowed stream 916.59


# uniform sample

In [136]:
sRate = 0.1
samples = np.random.choice(stream, size=int(w*sRate))
samplesNorm = np.linalg.norm(samples, ord=2)
uniformL2 = np.sqrt(samplesNorm**2 / sRate)

In [137]:
print('uniform L2 of {:0.2f} sampling rate: {:0.2f}'.format(samplingRate, uniformL2))

uniform L2 of 0.10 sampling rate: 941.34


# Sketch Sampling

In [80]:
from util import *

In [81]:
c=100
r=3
d, k=None, None
device='cuda'
streamTr=torch.tensor(stream, dtype=torch.int64)
item=streamTr[2]
streamTr0=streamTr[:10]

In [87]:
table = torch.rand(4,6)

In [88]:
table

tensor([[0.2033, 0.3602, 0.1523, 0.5134, 0.5653, 0.1703],
        [0.6022, 0.3949, 0.9913, 0.2097, 0.7614, 0.1823],
        [0.9457, 0.5329, 0.1083, 0.2407, 0.1977, 0.8877],
        [0.9612, 0.6931, 0.6629, 0.9909, 0.2194, 0.0885]])

In [84]:
p

In [85]:
aaa = lambda x: torch.norm(x, p=p, dim=1, keepdim=False, out=None)

In [89]:
aaa(table)

tensor([0.8980, 1.4690, 1.4405, 1.6976])

In [78]:
run(streamTr, c,r,device)

ERROR:root:File `'(streamTr,.py'` not found.


In [79]:
norms = [6898.5684, 4550.4961, 3416.7515, 2212.8689, 1666.8638, 1039.7496,
         748.4566,  470.0509,  355.3170,  192.5131,  125.5543, 66.5305,
          45.2503,   25.3738,   16.8966,    9.9995,    7.0583]

In [82]:
def get_windowed_id(norms, w):
    ids = np.array([])
    for norm in norms:
        ids  = np.append(ids, norm)
    return ids

In [123]:
aa = get_windowed_id(norms, w)

In [125]:
np.argsort(abs(aa- 475))[:2]

array([7, 8])

In [94]:
aa-aa[0]

array([ 0,  1, -1,  2,  3,  4,  5,  6,  7,  8,  9, -2, -3, -4, -5, -6, -7])

In [96]:
np.argsort(abs(aa - aa[0]))

array([ 0,  1,  2,  3, 11,  4, 12,  5, 13,  6, 14,  7, 15,  8, 16,  9, 10])

In [32]:
def create_csv(c,r, device):
    csv = CSNorm(c, r, device=device)
    return csv

In [35]:
def update_norm(csv, item):
    csv.accumulateItem(item)
    norm = csv.get_norm()
    return norm

In [36]:
def update_norms(csvs, c,r, device, item):
    csv0 = create_csv(c,r,device)
    csvs.append(csv0)
    norms = torch.tensor([], device = device)
    for csv in csvs:
        norm = update_norm(csv, item)    
        norms = torch.cat((norms, norm.view(1)), 0)        
    return norms

In [37]:
csvs = [csv]
norms = update_norms(csvs,c,r, device,streamTr[2])

In [119]:
def update_item(csvs, item, c,r,device):
    norms = update_norms(csvs, c,r, device, item)
    idxs = update_sketchs(norms)
#     print(idxs)
    csvsLeft = [csvs[i] for i in idxs]
    del csvs
    normsLeft = norms[list(idxs)]
    print(normsLeft)
    return csvsLeft, normsLeft 

In [120]:
def update_sketchs(norms):
    l= len(norms)
    if l <3: return list(range(l))
    result = set([])
    i = 0
    while i < l:
        result.add(i)
        found = False
        j = i+1
        while j < l:
            if norms[j] > norms[i]:
                i = j -1
                found = True
                break
            if (norms[j] < (norms[i] / 2.0)):
                if j != i+1:
                    result.add(j-1)                    
                i = j - 1
                found = True
                break;
            j+=1
        if not found and i != l-1: 
            result.add(l-1)
            return result
        i+=1
    return result

In [121]:
# def update_sketchs(norms):
#     l= len(norms)
#     if l <3: return list(range(l))
#     result = []
#     i = 0
#     while i < l:
#         result.append(i)
#         found = False
#         j = i+1
#         while j < l:
#             if norms[j] > norms[i]:
#                 i = j -1
#                 found = True
#                 break
#             if (norms[j] < (norms[i] / 2.0)):
#                 if j != i+1:
#                     result.append(j-1)                    
#                 i = j - 1
#                 found = True
#                 break;
#             j+=1
#         if not found and i != l-1: 
#             result.append(l-1)
#             return result
#         i+=1
#     return result

In [122]:
def run(streamTr, c,r,device):
    csvs = []
    for i in range(len(streamTr)):
        csvs, norms = update_item(csvs, streamTr[i], c,r,device)
    return norms

In [123]:
run(streamTr, c, r, device)

tensor([7.0583], device='cuda:0')
tensor([10.3368,  7.0679], device='cuda:0')
tensor([12.3282,  7.0583], device='cuda:0')
tensor([13.9998,  7.0679], device='cuda:0')
tensor([15.2962,  9.9995,  7.0707], device='cuda:0')
tensor([16.5551, 10.0990,  7.0707], device='cuda:0')
tensor([17.9938,  9.8990,  7.0707], device='cuda:0')
tensor([18.6862,  9.5894,  7.0707], device='cuda:0')
tensor([19.4651,  9.8949,  7.0679], device='cuda:0')
tensor([22.5514, 14.1358,  7.0679], device='cuda:0')
tensor([23.4135, 15.4092,  9.6816,  7.0679], device='cuda:0')
tensor([24.8781, 12.7124, 10.2395,  7.0654], device='cuda:0')
tensor([25.4337, 14.6603, 12.5698,  7.0536], device='cuda:0')
tensor([26.5432, 13.6746,  9.3754,  7.0679], device='cuda:0')
tensor([28.2356, 16.2268, 12.1611,  7.0353], device='cuda:0')
tensor([28.2168, 17.9357, 14.0340,  7.0622], device='cuda:0')
tensor([31.0121, 16.3697, 10.2791,  7.0679], device='cuda:0')
tensor([31.5767, 17.4596, 12.2045,  7.0622], device='cuda:0')
tensor([31.7520, 18.

tensor([100.6223,  54.5115,  38.9238,  21.2124,  13.9641,   7.0707],
       device='cuda:0')

In [ ]:
def find_set(array):
    result = set()
    i = 0
    while i < len(array):
        print('loop',i,result)
        set.append(array[i])
        found = False
        j = i+1
        while j < len(array):
            print(i, j)
            if array[j] > array[i]:
                i = j -1
                found = True
                break
            if (array[j] < (array[i] / 2)):
                if j != i+1:
                    result.append(array[j-1])

                print(result)
                i = j - 1
                found = True
                break;
            j+=1
        if not found: 
            result.append(array[-1])
            return result
        i+=1
    return result

In [ ]:
def find(array):
    result = []
    i = 0
    while i < len(array):
        print('loop',i,result)
        result.append(array[i])
        found = False
        j = i+1
        while j < len(array):
            print(i, j)
            if array[j] > array[i]:
                i = j -1
                found = True
                break
            if (array[j] < (array[i] / 2)):
                if j != i+1:
                    result.append(array[j-1])

                print(result)
                i = j - 1
                found = True
                break;
            j+=1
        if not found: 
            result.append(array[-1])
            return result
        i+=1
    return result